# Introduction to Geospatial AI

# Intro
Welcome to this workship about geospatial AI! In this workshop you will try to detect buildings from aerial images. This is done in three steps;

1. Creating training data.
2. Training machine learning models using the created training data.
3. Evaluating the trained models and predicting where buildings are located in images the models haven't seen before. 

We will be using jupyter notebooks with Google Colab, but you don't need to have any experience with these in order to complete this workshop.

# Task 0.0 - Create copy of notebook and select GPU env

But first, before we can do any of the fun stuff, we need to set up the environment properly. In order to do that follow the steps under;

1. Create a copy of the notebook in your drive by saving it. Shortcut is `Ctrl + s` on Windows or `Cmd + s` on Mac. This opens the notebook in another tab.
2. Change to a GPU runtime environment. In the top right corner choose `Change runtime type` and select `T4 GPU`.

Nice! Next, we need to clone the git repo we are working with and adding it to the path in addition to installing some dependencies. To do this simply run the cell below.

In [ ]:
!git clone https://github.com/kartAI/kartAI.git
!git -C /content/kartAI/ checkout workshop

!pip install focal_loss
!pip install azure-storage-blob
!pip install rasterio
!pip install rasterstats
!pip install colour

# Task 0.1 - Restart runtime

Sometimes the runtime can be quite slow when cloning repos and switching to different branches. In order to get these changes in, we need to restart the runtime. Here is how to do it;

1. Press on the `Runtime` tab in the menu at the top left.
2. Press `Restart runtime`. Check in the top right corner that you are connected to a T4 GPU.

That's it! We are ready to start. Run the next cell to add the github repo to the path of the VM this notebook is running on.

In [ ]:
import sys
sys.path.insert(0,'/content/kartAI')

# Task 1.1 - Select training dataset name and area
In the first task to do is selecting an area you want to train on, then download aerial photos in addition to data about all the existing buildings in the chosen area. 

In the next cell, choose a name for your dataset and which of the 3 areas you want to train from. You can also define your own area. Head over to https://geojson.io/ to find coordinates and train your model on your custom area. NB!! We dont have a dataset over all of norway, so you might have missing data if you chose an area that's not covered. Also, you need to convert the coordinates to a different coordinate system...

In [ ]:
# Set the name of your training set
training_dataset_name = "best_training_dataset"

# This area covers Sand, outside of Jessheim
area = { "x_min": 618296.0, "x_max": 620895.0, "y_min": 6668145.0, "y_max": 6670133.0 }
centre_of_training_area = [619000.0, 6669500]

# This area covers Skøyen, Oslo
# area = { "x_min": 593150.9, "x_max": 596528.0, "y_min": 6643812.3, "y_max": 6644452.2 }
# centre_of_training_area = [593150.9, 6644452.2]

# This area covers Midtbyen, Trondheim
# area = { "x_min": 568372.6, "x_max": 570820.4, "y_min": 7033216.7, "y_max": 7034223.7 }
# centre_of_training_area = [568372.6, 7034223.7]

# Task 1.2 - Create training data
When you have chosen the area you can run the cell after in order to create the training data. The training data is downloaded as rasters (images) and split into a training, validation and test set. The model will train on the training set, and while training run tests on the validation set. After the training is finished it will run tests on the tests set - data the model have never seen before.

While downloading the rasters, it will say how many rasters total it will download. The training data download is quite time consuming, so if you make a custom area, make sure it downloads maximum about ~700 rasters.

In [ ]:
from kartAI.kartai.tools.create_training_data import create_training_data

# If you have already created a dataset and want to create a new one, you should wipe the old dataset by setting this flag to True.
wipe_training_data = False

if wipe_training_data:
    dir = "/content/training_data/OrtofotoWMS/25832_563000.0_6623000.0_100.0_100.0/512/" # Make sure this is the path to the downloaded dataset
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))

create_training_data(
    training_dataset_name=training_dataset_name, 
    config_file_path="kartAI/config/dataset/osm_bygg.json",
    eager_load=True,
    confidence_threshold=None, 
    eval_model_checkpoint=None,
    region=None, 
    x_min=area["x_min"], 
    x_max=area["x_max"], 
    y_min=area["y_min"], 
    y_max=area["y_max"],
    num_processes=None                 
)

# Task 1.3 - Visualize training data
After downloading the data you can visualize it in the next cell. Make sure the path to the training data is correct.

In [ ]:
import folium
import rasterio
import os

from pyproj import CRS, Transformer

path_to_dir = "/content/training_data/OrtofotoWMS/25832_563000.0_6623000.0_100.0_100.0/512/" # Make sure to have the correct path to the training data.
files = os.listdir(path_to_dir)
files.sort()

crs_25832 = CRS.from_epsg(25832)
crs_4326 = CRS.from_epsg(4326)
transformer = Transformer.from_crs(crs_25832, crs_4326)

fig = folium.Figure(width=800, height=400)
m = folium.Map(
    location=transformer.transform(centre_of_training_area[0], centre_of_training_area[1]), 
    zoom_start=14
)

for i in range(5): # Load 5 rasters. Change this to load fewer/more rasters. Be aware that loading many rasters is slow.
    with rasterio.open(f"{path_to_dir}{files[i]}") as src:
        img = src.read()
        transformed_bottom_left = transformer.transform(src.bounds.left, src.bounds.bottom)
        transformed_top_right = transformer.transform(src.bounds.right, src.bounds.top)
    m.add_child(folium.raster_layers.ImageOverlay(img.transpose(1, 2, 0), bounds = [transformed_bottom_left, transformed_top_right]))

fig.add_child(m)

# Task 2.1 - Choose model type and hyperparameters
After creating and visualizing the training data we are ready to train our model! Before we start training the model we need to decide what kind of model you want to build. Give your model a name and choose hyperparameters you want to work with.

The `model` argument below decides what kind of model architecture you want to use. The supported ones are 
 - `unet`. The “U-Net” architecture consists of 2 parts: the first part is a “classic” Convolutional Neural Network which scans the image, extract patterns from it, and combine them into high resolutions features. Then, the network is asked to upscale its hidden layers into recreating a full binary image, where each pixel is either 0 or 1.
 - `resnet`. Residual Network (ResNet) architecture is a type of artificial neural network that allows the model to skip layers without affecting performance and therefore circumventing the vanishing gradient problem.
 - `CSP`. A Cross Stage Partial (CSP) block is designed to attribute the problem of duplicate gradient information within network optimization. The input is split into two paths, where one path is goes through a dense block, while the other skips the block and joins at the end. This means complexity can be reduced while maintaining the accuracy.

If you want to know more about the models and their architecture feel free to ask us or google them!

In [ ]:
# Choose model name and architecture
model_name = "super_ultra_wow_model"
model_architecture = "unet"

train_args = {
      "features": 32,
      "depth": 4,
      "optimizer": "RMSprop",
      "batch_size": 8,
      "model": model_architecture,
      "loss": "binary_crossentropy",
      "activation": "relu",
      "epochs": 20
}

# Task 2.2 - Train machine learning model
The training performs what is known as per-pixel classification. In other words, the model tries to assign a class (either a building or not a building) for each pixel in the raster based on the input features. After the model is trained we can create vector data from the predicted pixels and therefore end up with bounding boxes we can look at!

In the next cell you can tune some hyperparameters, but make sure the training doesn't take too long. The default configuration should take about ~15 minutes to execute and should get you a _decent_ model.

While training some statistics about the training is showing. They can be a little bit confusing, and it's not a must to understand all of them. The stats showing are;

 - Loss: A measurement of how wrong the model is. The lower the loss is, the better. If the loss is 0, the model is "perfect". A model tries to minimize this value.
 - Binary Accuracy: A measurement of how many of the predicted pixels are inside a building. It's a number between 0 and 1, where higher is better. 1 means all the pixels the model says are within a building is actually within a building. But keep in mind even if the number is 1, the model might not have made predictions for all pixels in all buildings...
 - IoU: Intersection over Union. A measurement of how much of the estimated area overlaps with a building. It's a number between 0 and 1, where higher is better. 1 means the model is fitting the bounding box of all buildings "perfectly".
 - IoU_fz: Fuzzy set variant of IoU. Shares similar characteristics as described earlier.
 - IoU_point_[5-9]: Cutoff values for IoU. It's a measurement of what the IoU would be if the cutoff values was [5-9].
 - val_x: The validation equivalent of whatever x is. X could be loss, IoU, etc.

In [ ]:
from kartAI.kartai.tools.train import train

train(
      checkpoint_name=model_name,
      dataset_name=[training_dataset_name],
      input_generator_config_path="kartAI/config/ml_input_generator/ortofoto.json",
      save_model=False,
      train_args=train_args,
      checkpoint_to_finetune=False
)


# Task 3.1 - Running inference on test set
For task 3 we will use our trained machine learning model and try to find buildings in a new set of images we haven't seen so far. The next cell runs predictions on the test portion of the downloaded training data. The same statistics as the ones described during training shows up, in addition to;

 - Confidence: Tells us how confident the model is. It is a number between 0 and 1, where higher is better.

In [ ]:
import os
import json
from kartAI.env import get_env_variable
from kartAI.kartai.tools.predict import predict_and_evaluate

created_datasets_dir = os.path.join(get_env_variable(
    'created_datasets_directory'), training_dataset_name)

checkpoint_path = os.path.join(get_env_variable(
    'trained_models_directory'), f'{model_name}.h5')

with open("kartAI/config/ml_input_generator/ortofoto.json", encoding="utf8") as config:
    datagenerator_config = json.load(config)

predict_and_evaluate(
    created_datasets_path=created_datasets_dir,
    datagenerator_config=datagenerator_config,
    checkpoint_name_to_predict_with=model_name,
    save_prediction_images=True,
    save_diff_images=True,
    generate_metadata=True,
    dataset_to_evaluate="test",
    batch_size=200
)

# Task 3.2 - Choose a test area
Now that we have looked at some stats from the predictions, let's look at some images! First, we need to set a name and the coordinates of the centre for the test region. 

We have set up three different areas to look at:
- Kristiansand.
- Stavanger.
- Bodø.

If you want to set up a completely custom area to look at you have to do the following;

1. Create a new file in kartAi/training_data named `<YOUR_NAME_FOR_AREA>.json`
2. Find the area you want to look at, for example at `geojson.io`
3. Transform the coordinates from EPSG4326 to EPSG25832, for example at https://epsg.io/transform#s_srs=4326&t_srs=25832
4. Plot these new coorinates in the file created at step 1. Look at the other files in the same directory to look at examples.
5. Restart runtime environment.
6. Rerun the cells with local variables.

Ask us for help if you want to set up a custom area :-)

In [ ]:
# Set up test regions here. Choose your preferred area

# Kristiansand
test_region_name = "kristiansand"
centre_of_prediction_area = [441069.2, 6447433.6]

# Stavanger
# test_region_name = "stavanger"
# centre_of_prediction_area = [312311.63370744896, 6541187.02137879]

# Bodø
# test_region_name = "bodo"
# centre_of_prediction_area = [731181.2714365568, 7472141.840146479]

# Task 3.3 - Create vector data
In the next cell we download a different set of rasters and perform predictions on these. After the predictions are made, we create vector data based on the predictions. The vector data generated can be used to visualize our predictions in a map to see how the model is performing.

In [ ]:
from kartAI.kartai.dataset.create_polygon_dataset import produce_vector_dataset, run_ml_predictions
from kartAI.kartai.utils.config_utils import read_config
from kartAI.kartai.utils.crs_utils import get_projection_from_config_path
from kartAI.kartai.utils.geometry_utils import parse_region_arg
from kartAI.kartai.utils.prediction_utils import get_raster_predictions_dir, get_vector_predictions_dir

geom = parse_region_arg(f"kartAI/training_data/regions/{test_region_name}.json")

projection = get_projection_from_config_path("kartAI/config/dataset/osm_bygg.json")

config = read_config("kartAI/config/dataset/osm_bygg.json")

run_ml_predictions(
    input_model_name=model_name, 
    region_name=test_region_name, 
    projection=projection,
    config=config, 
    geom=geom, 
    batch_size=200, 
    skip_data_fetching=False,
    save_to="local", 
    num_processes=1
)

vector_output_dir = get_vector_predictions_dir(test_region_name, model_name)
raster_predictions_path = get_raster_predictions_dir(test_region_name, model_name)

produce_vector_dataset(
    output_dir=vector_output_dir, 
    raster_dir=raster_predictions_path, 
    config=config, 
    max_batch_size=200, 
    modelname=f"{test_region_name}_{model_name}", 
    save_to="local"
)

# Task 3.4 - Visualize vector data
The next cell visualizes the created vector data in a map.

In [ ]:
import folium
import geopandas as gp

polygon_25832 = gp.read_file(f"results/{test_region_name}/{model_name}/vector/raw_predictions_0.json")
polygon_4326 = polygon_25832.to_crs(4326)

fig = folium.Figure(width=800, height=400)
map = folium.Map(location=transformer.transform(centre_of_prediction_area[0], centre_of_prediction_area[1]), zoom_start=14)
folium.GeoJson(data=polygon_4326["geometry"]).add_to(map)
fig.add_child(map)

# Task 3.5 - Create contour data
Lastly, we create contours. These are similar to the vector data but we can set a cutoff value on how confident the model is that the given area is a building or not. I.e. a contour value of 0.3 means that it will create a polygon around pixels that the model is at least 30% sure is a building, while a contour value of 0.8 creates a polygon around all pixels that the model is 80% sure is a building.

In [ ]:
from kartAI.kartai.dataset.create_polygon_dataset import run_ml_predictions
from kartAI.kartai.tools.predict import create_contour_result
from kartAI.kartai.utils.config_utils import read_config
from kartAI.kartai.utils.crs_utils import get_projection_from_config_path
from kartAI.kartai.utils.geometry_utils import parse_region_arg
from kartAI.kartai.utils.prediction_utils import get_contour_predictions_dir, get_raster_predictions_dir

geom = parse_region_arg(f"kartAI/training_data/regions/{test_region_name}.json")

projection = get_projection_from_config_path("kartAI/config/dataset/osm_bygg.json")

config = read_config("kartAI/config/dataset/osm_bygg.json")

run_ml_predictions(
    input_model_name=model_name,
    region_name=test_region_name,
    projection=projection,
    config=config,
    geom=geom,
    batch_size=200,
    skip_data_fetching=False,
    save_to="local",
    num_processes=1
)

raster_output_dir = get_raster_predictions_dir(test_region_name, model_name)
contour_output_dir = get_contour_predictions_dir(test_region_name, model_name)

print("---> Creating contour dataset from rasters")

contour_levels = [0.5, 0.8] # Change contour levels here.
create_contour_result(raster_output_dir, contour_output_dir, projection, contour_levels)

print("==== Contour dataset created ====")


# Task 3.6 - Visualize contour data
The last cell visualizes the contours we generated.

In [ ]:
import folium
import geopandas as gp
import math
from colour import Color

contour_25832 = gp.read_file(f"results/{test_region_name}/{model_name}/contour/complete_contour.json")
contour_25832["geometry"] = contour_25832.simplify(tolerance=1)
contour_4326 = contour_25832.to_crs(4326)

figure = folium.Figure(width=800, height=400)
map = folium.Map(location=transformer.transform(centre_of_prediction_area[0], centre_of_prediction_area[1]), zoom_start=15)
folium.GeoJson(
    data=contour_4326, 
    style_function=lambda x: { "color": list(Color("white").range_to(Color("red"), 10))[math.floor(x["properties"]["elev"]*10)].hex }
    ).add_to(map)
figure.add_child(map)